In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

# Definizione funzioni

In [ ]:
#Legge per la figura di interferenza di Fabry-Perot

def FP_maximum(N, lambda_, d, delta):
    x = np.arccos(lambda_/(2*d)*(N-delta/(2*np.pi))) #angolo di incidenza
    return x

#Legge per calibrazione del nonio in Fabry-Perot

def FB_nonio(dN, lambda_, theta, theta_err):
    x = dN*lambda_/(2*np.cos(theta))    #variazione di distanza tra le lastre
    y = dN*lambda_*np.sin(theta)*theta_err/(2*np.cos(theta)**3)    
    return x, y